# After realizing dummy mistake

JSON to CSV/Dataframe conversion

In [26]:
import json
import pandas as pd
import os
import traceback

def safe_extract_value(data, *keys, default=None):
    """
    Safely extract values from nested dictionaries, handling both direct values and {value, location, reasoning} structures.
    
    Parameters:
    -----------
    data : dict
        Dictionary to extract value from
    *keys : list
        Keys to navigate through the nested dictionary
    default : any
        Default value to return if key doesn't exist
        
    Returns:
    --------
    The extracted value or default
    """
    try:
        current = data
        for key in keys[:-1]:
            if isinstance(current, dict) and key in current:
                current = current[key]
            else:
                return default
        
        # Last key processing
        last_key = keys[-1]
        if isinstance(current, dict) and last_key in current:
            item = current[last_key]
            
            # Handle both direct values and {value, location, reasoning} structures
            if isinstance(item, dict) and 'value' in item:
                return item['value']
            else:
                return item
        else:
            return default
    except Exception:
        return default

def get_json_files(directory='docs'):
    """
    Returns a list of all JSON files in the specified directory.
    
    Parameters:
    -----------
    directory : str
        Path to the directory to search (default: 'docs')
        
    Returns:
    --------
    list
        List of full file paths to all JSON files in the directory
    """
    json_files = []
    
    # Check if the directory exists
    if not os.path.exists(directory):
        print(f"Directory '{directory}' does not exist.")
        return json_files
    
    # Walk through the directory and find all JSON files
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.json'):
                # Get full path
                file_path = os.path.join(root, file)
                json_files.append(file_path)
    
    print(f"Found {len(json_files)} JSON files in {directory}.")
    return json_files

def process_json_files(file_paths_or_path):
    """
    Process one or multiple JSON files and convert them into a structured DataFrame.
    
    Parameters:
    -----------
    file_paths_or_path : list or str
        List of file paths to the JSON files or a single file path
        
    Returns:
    --------
    pd.DataFrame
        DataFrame containing the extracted data from all JSON files
    """
    # List to store data from each file
    all_data = []
    
    # Handle different input types
    if isinstance(file_paths_or_path, str):
        file_paths = [file_paths_or_path]
    else:
        file_paths = file_paths_or_path
    
    # Process each file
    for file_path in file_paths:
        try:
            # Load JSON data
            with open(file_path, 'r', encoding='utf-8') as f:
                file_content = f.read()
                data = json.loads(file_content)
            
            # Extract country name
            country = safe_extract_value(data, 'submission_metadata', 'country', default='Unknown')
            print(f"Processing country: {country}")
            
            # Create a dictionary for this country's data
            country_data = {'country': country}
            
            # Extract objectives data
            # C1: End plastic pollution
            end_pollution_mentioned = safe_extract_value(data, 'objectives', 'end_plastic_pollution', 'mentioned', default=False)
            end_pollution_timeframe = safe_extract_value(data, 'objectives', 'end_plastic_pollution', 'timeframe_specified', default=False)
            
            country_data['A : C1 Objectives - end plastic pollution'] = end_pollution_mentioned
            country_data['B : Mentioned with time frame'] = end_pollution_mentioned and end_pollution_timeframe
            country_data['C : Mentioned, no time frame'] = end_pollution_mentioned and not end_pollution_timeframe
            country_data['D : Not mentioned'] = not end_pollution_mentioned
            
            # C2: Reduce production of plastics
            reduce_production_mentioned = safe_extract_value(data, 'objectives', 'reduce_production', 'mentioned', default=False)
            reduce_production_spec = safe_extract_value(data, 'objectives', 'reduce_production', 'specification_provided', default=False)
            
            country_data['E : C2 Objectives - reduce production of plastics'] = reduce_production_mentioned
            country_data['F : Mentioned with specification'] = reduce_production_mentioned and reduce_production_spec
            country_data['G : Mentioned, no specification'] = reduce_production_mentioned and not reduce_production_spec
            country_data['H : Not mentioned'] = not reduce_production_mentioned
            
            # C3: Benefits of plastics
            benefits_mentioned = safe_extract_value(data, 'objectives', 'benefits_of_plastics', 'mentioned', default=False)
            
            country_data['I : C3 Objectives - benefits of plastics'] = benefits_mentioned
            country_data['J : Mentioned'] = benefits_mentioned
            country_data['K : Not mentioned'] = not benefits_mentioned
            
            # C4: Protect human health
            health_mentioned = safe_extract_value(data, 'objectives', 'protect_human_health', 'mentioned', default=False)
            
            country_data['L : C4 Objectives - protect human health'] = health_mentioned
            country_data['M : Mentioned'] = health_mentioned
            country_data['N : Not mentioned'] = not health_mentioned
            
            # C5: Protect biodiversity
            biodiversity_mentioned = safe_extract_value(data, 'objectives', 'protect_biodiversity', 'mentioned', default=False)
            
            country_data['O : C5 Objectives - protect biodiversity and (marine) environment'] = biodiversity_mentioned
            country_data['P : Mentioned'] = biodiversity_mentioned
            country_data['Q : Not mentioned'] = not biodiversity_mentioned
            
            # C10: Time horizon of implementation
            timeframe_specified = safe_extract_value(data, 'implementation', 'timeframe', 'specified', default=False)
            
            country_data['R : C10 Time horizon of implementation'] = True
            country_data['S : Not relevant'] = False
            country_data['T : Not specified'] = not timeframe_specified
            country_data['U : Specified'] = timeframe_specified
            
            # C11: Stringency of measure
            stringency_level = safe_extract_value(data, 'implementation', 'stringency', 'level', default='')
            
            country_data['V : C11 Stringency of measure'] = True
            country_data['W : High'] = stringency_level == 'High'
            country_data['X : Low'] = stringency_level == 'Low'
            country_data['Y : Non relevant'] = stringency_level == ''
            
            # C6: Addressing the full life cycle of plastics
            lifecycle_mentioned = safe_extract_value(data, 'objectives', 'lifecycle_approach', 'mentioned', default=False)
            lifecycle_coverage = safe_extract_value(data, 'objectives', 'lifecycle_approach', 'coverage', default='')
            
            country_data['Z : C6 Objectives - addressing the full life cycle of plastics'] = lifecycle_mentioned
            country_data['AA : Mentioned'] = lifecycle_mentioned and lifecycle_coverage == 'Full lifecycle'
            country_data['AB : Not mentioned'] = not lifecycle_mentioned
            country_data['AC : Partial mention'] = lifecycle_mentioned and lifecycle_coverage != 'Full lifecycle'
            
            # C7: Other objectives
            other_objectives = safe_extract_value(data, 'objectives', 'other_objectives', default=[])
            # Make sure it's a list even if a dict was returned
            if not isinstance(other_objectives, list):
                other_objectives = []
                
            has_other_objectives = len(other_objectives) > 0
            
            # Extract specific objectives - safely handle if items are dicts with 'value' keys
            def check_if_contains(obj_list, keyword):
                for obj in obj_list:
                    value = obj.get('value', obj) if isinstance(obj, dict) else obj
                    if isinstance(value, str) and keyword in value.lower():
                        return True
                return False
            
            has_circular_economy = check_if_contains(other_objectives, 'circular')
            has_climate_change = check_if_contains(other_objectives, 'climate')
            has_esm = check_if_contains(other_objectives, 'sound management')
            has_sustainable_production = check_if_contains(other_objectives, 'sustainable production')
            
            country_data['AD : C7 Objectives - other objectives'] = has_other_objectives
            country_data['AE : Circular economy'] = has_circular_economy
            country_data['AF : Climate change'] = has_climate_change
            country_data['AG : ESM'] = has_esm
            country_data['AH : Mentioned'] = has_other_objectives
            country_data['AI : Not mentioned'] = not has_other_objectives
            country_data['AJ : Sustainable production'] = has_sustainable_production
            
            # C8: Value chain
            upstream_feedstock = safe_extract_value(data, 'value_chain', 'upstream', 'feedstock', 'mentioned', default=False)
            upstream_production = safe_extract_value(data, 'value_chain', 'upstream', 'production', 'mentioned', default=False)
            upstream_mentioned = upstream_feedstock or upstream_production
            
            midstream_design = safe_extract_value(data, 'value_chain', 'midstream', 'design', 'mentioned', default=False)
            midstream_product = safe_extract_value(data, 'value_chain', 'midstream', 'product_production', 'mentioned', default=False)
            midstream_distribution = safe_extract_value(data, 'value_chain', 'midstream', 'distribution', 'mentioned', default=False)
            midstream_consumption = safe_extract_value(data, 'value_chain', 'midstream', 'consumption', 'mentioned', default=False)
            midstream_mentioned = midstream_design or midstream_product or midstream_distribution or midstream_consumption
            
            downstream_collection = safe_extract_value(data, 'value_chain', 'downstream', 'collection', 'mentioned', default=False)
            downstream_waste = safe_extract_value(data, 'value_chain', 'downstream', 'waste_management', 'mentioned', default=False)
            downstream_recycling = safe_extract_value(data, 'value_chain', 'downstream', 'recycling', 'mentioned', default=False)
            downstream_legacy = safe_extract_value(data, 'value_chain', 'downstream', 'legacy_plastic', 'mentioned', default=False)
            downstream_mentioned = downstream_collection or downstream_waste or downstream_recycling or downstream_legacy
            
            cross_emissions = safe_extract_value(data, 'value_chain', 'cross_value_chain', 'emissions', 'mentioned', default=False)
            cross_microplastic = safe_extract_value(data, 'value_chain', 'cross_value_chain', 'microplastic_leakage', 'mentioned', default=False)
            cross_mentioned = cross_emissions or cross_microplastic
            
            country_data['AK : C8 Value chain'] = True
            country_data['AL : 1. Upstream'] = upstream_mentioned
            country_data['AM : 2. Midstream'] = midstream_mentioned
            country_data['AN : 3. Downstream'] = downstream_mentioned
            country_data['AO : 4. Cross value chain'] = cross_mentioned
            
            # C9: Type of measure
            has_targets = safe_extract_value(data, 'measures', 'targets', 'present', default=False)
            
            # Check for instruments - safely with checking dict paths
            def has_any_instruments(data, category, instrument_keys):
                if not isinstance(data, dict) or 'measures' not in data:
                    return False
                measures = data['measures']
                if not isinstance(measures, dict) or category not in measures:
                    return False
                category_data = measures[category]
                if not isinstance(category_data, dict):
                    return False
                    
                for key in instrument_keys:
                    if key in category_data:
                        instr_data = category_data[key]
                        if isinstance(instr_data, dict) and 'mentioned' in instr_data:
                            mentioned = instr_data['mentioned']
                            if isinstance(mentioned, dict) and 'value' in mentioned:
                                if mentioned['value']:
                                    return True
                            elif mentioned:
                                return True
                return False
            
            economic_instrument_keys = ['tax_incentives', 'subsidies', 'penalties', 'trading_systems', 
                                       'deposit_systems', 'public_procurement', 'rd_funding']
            economic_instruments = has_any_instruments(data, 'economic_instruments', economic_instrument_keys)
            
            regulatory_instrument_keys = ['bans', 'moratoriums', 'performance_standards', 'mandatory_infrastructure',
                                         'certification', 'labelling', 'action_plans', 'reporting', 
                                         'trade_requirements', 'epr', 'just_transition']
            regulatory_instruments = has_any_instruments(data, 'regulatory_instruments', regulatory_instrument_keys)
            
            soft_instrument_keys = ['voluntary_certification', 'voluntary_labelling', 'monitoring', 
                                    'information_guidance', 'education', 'expert_groups', 'research_promotion', 
                                    'harmonization', 'knowledge_sharing']
            soft_instruments = has_any_instruments(data, 'soft_instruments', soft_instrument_keys)
            
            has_instruments = economic_instruments or regulatory_instruments or soft_instruments
            
            country_data['AP : C9 Type of measure'] = has_instruments or has_targets
            country_data['AQ : Instrument'] = has_instruments
            country_data['AR : Target'] = has_targets
            
            # Add data to our list
            all_data.append(country_data)
            print(f"Successfully processed {country}")
            
        except json.JSONDecodeError as e:
            print(f"JSON parsing error in file {file_path}: {e}")
            print(f"First 100 characters of file: {file_content[:100] if 'file_content' in locals() else 'Not available'}")
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
            traceback.print_exc()
    
    # Convert to DataFrame
    if all_data:
        df = pd.DataFrame(all_data)
        return df
    else:
        print("No data was successfully processed.")
        return pd.DataFrame()

def run_extraction(directory='docs', output_file='country_submissions_analysis.xlsx'):
    """
    Run the extraction process on all JSON files in a directory and save results to Excel.
    
    Parameters:
    -----------
    directory : str
        Path to the directory containing JSON files
    output_file : str
        Path to the output Excel file
    
    Returns:
    --------
    pd.DataFrame
        DataFrame containing the extracted data
    """
    print(f"Starting extraction process on directory: {directory}")
    
    # Get all JSON files
    json_files = get_json_files(directory)
    
    if not json_files:
        print("No JSON files found. Cannot proceed.")
        return None
    
    # Process all JSON files
    df = process_json_files(json_files)
    
    if df is not None and not df.empty:
        # Save to Excel
        df.to_excel(output_file, index=False)
        print(f"Results saved to {output_file}")
        
        # Print some statistics
        print(f"\nExtraction Results:")
        print(f"Total countries processed: {len(df)}")
        print(f"Columns in output: {len(df.columns)}")
    
    return df

result_df = run_extraction('docs', 'country_submissions_analysis.xlsx')
    
# Display the first few rows of the result
if result_df is not None and not result_df.empty:
    print("\nFirst 5 rows of the extracted data:")
    print(result_df.head(10))

Starting extraction process on directory: docs
Found 11 JSON files in docs.
Processing country: United States of America
Successfully processed United States of America
Processing country: Bosnia and Herzegovina
Successfully processed Bosnia and Herzegovina
Processing country: Saudi Arabia
Successfully processed Saudi Arabia
Processing country: Principality of Monaco
Successfully processed Principality of Monaco
Processing country: Islamic Republic of Iran
Successfully processed Islamic Republic of Iran
Processing country: Cambodia
Successfully processed Cambodia
Processing country: Brazil
Successfully processed Brazil
Processing country: The Russian Federation
Successfully processed The Russian Federation
Processing country: URUGUAY
Successfully processed URUGUAY
Processing country: COOK ISLANDS
Successfully processed COOK ISLANDS
Processing country: European Union
Successfully processed European Union
Results saved to country_submissions_analysis.xlsx

Extraction Results:
Total count

In [27]:
check_LLM = pd.read_excel('country_submissions_analysis.xlsx')
check_LLM.country

0     United States of America
1       Bosnia and Herzegovina
2                 Saudi Arabia
3       Principality of Monaco
4     Islamic Republic of Iran
5                     Cambodia
6                       Brazil
7       The Russian Federation
8                      URUGUAY
9                 COOK ISLANDS
10              European Union
Name: country, dtype: object

In [28]:
check_nvivo = pd.read_csv('nvivo_export.csv').rename(columns={'Unnamed: 0': 'country'})
check_nvivo.country

0                         1 : EU
1     2 : Bosnia and Herzegovina
2                     3 : Brazil
3                   4 : Cambodia
4               5 : Cook Islands
5                       6 : Iran
6                     7 : Monaco
7                     8 : Russia
8               9 : Saudi Arabia
9                   10 : Uruguay
10                      11 : USA
Name: country, dtype: object

## Rename countries in LLM dataframe to match Nvivo

In [29]:
def map_country_names(df_llm):
    """
    Maps country names in the LLM dataframe to match the Nvivo format.
    
    Parameters:
    -----------
    df_llm : pandas.DataFrame
        Dataframe containing LLM-generated data with country names
        
    Returns:
    --------
    pandas.DataFrame
        A copy of the dataframe with updated country names
    """
    # Create a dictionary mapping LLM country names to Nvivo format
    country_mapping = {
        'United States of America': '11 : USA',
        'Bosnia and Herzegovina': '2 : Bosnia and Herzegovina',
        'Saudi Arabia': '9 : Saudi Arabia',
        'Principality of Monaco': '7 : Monaco',
        'Islamic Republic of Iran': '6 : Iran',
        'Cambodia': '4 : Cambodia',
        'Brazil': '3 : Brazil',
        'The Russian Federation': '8 : Russia',
        'URUGUAY': '10 : Uruguay',
        'COOK ISLANDS': '5 : Cook Islands',
        'European Union': '1 : EU'
    }
    
    # Create a copy of the original dataframe
    df_mapped = df_llm.copy()
    
    # Apply the mapping to the country column
    df_mapped['country'] = df_mapped['country'].map(country_mapping).fillna(df_mapped['country'])
    
    return df_mapped

In [30]:
llm_data = map_country_names(check_LLM)
llm_data

,country,A : C1 Objectives - end plastic pollution,B : Mentioned with time frame,"C : Mentioned, no time frame",D : Not mentioned,E : C2 Objectives - reduce production of plastics,F : Mentioned with specification,"G : Mentioned, no specification",H : Not mentioned,I : C3 Objectives - benefits of plastics,...,AI : Not mentioned,AJ : Sustainable production,AK : C8 Value chain,AL : 1. Upstream,AM : 2. Midstream,AN : 3. Downstream,AO : 4. Cross value chain,AP : C9 Type of measure,AQ : Instrument,AR : Target
0,11 : USA,True,True,False,False,True,False,True,False,True,...,True,False,True,True,True,True,True,True,True,False
1,2 : Bosnia and Herzegovina,True,False,True,False,True,True,False,False,True,...,False,False,True,True,True,True,True,True,True,False
2,9 : Saudi Arabia,True,False,True,False,True,False,True,False,True,...,True,False,True,False,True,True,False,True,True,True
3,7 : Monaco,True,True,False,False,True,True,False,False,False,...,True,False,True,True,True,True,True,True,True,True
4,6 : Iran,True,False,True,False,False,False,False,True,False,...,True,False,True,False,False,True,False,True,True,False
5,4 : Cambodia,True,True,False,False,True,False,True,False,False,...,True,False,True,True,True,True,False,True,True,False
6,3 : Brazil,True,False,True,False,True,False,True,False,False,...,True,False,True,True,True,True,False,True,True,False
7,8 : Russia,True,False,True,False,False,False,False,True,True,...,True,False,True,False,False,True,False,True,True,True
8,10 : Uruguay,True,False,True,False,True,True,False,False,False,...,True,False,True,True,True,True,True,True,True,True
9,5 : Cook Islands,True,False,True,False,True,True,False,False,False,...,True,False,True,True,True,True,True,True,True,True


## Reorder LLM dataframe to match row order in Nvivo data

In [31]:
def reorder_to_match(df_to_reorder, reference_df):
    """
    Reorders a dataframe to match the country order of a reference dataframe.
    
    Parameters:
    -----------
    df_to_reorder : pandas.DataFrame
        The dataframe to reorder
    reference_df : pandas.DataFrame
        The reference dataframe with the desired country order
        
    Returns:
    --------
    pandas.DataFrame
        The reordered dataframe
    """
    # Get the ordered list of countries from the reference dataframe
    country_order = reference_df['country'].tolist()
    
    # Create a categorical column with the specified order
    df_result = df_to_reorder.copy()
    df_result['country'] = pd.Categorical(df_result['country'], categories=country_order, ordered=True)
    
    # Sort by the categorical column and reset index
    result = df_result.sort_values('country').reset_index(drop=True)
    
    return result

In [32]:
llm_data = reorder_to_match(llm_data, check_nvivo)
llm_data

,country,A : C1 Objectives - end plastic pollution,B : Mentioned with time frame,"C : Mentioned, no time frame",D : Not mentioned,E : C2 Objectives - reduce production of plastics,F : Mentioned with specification,"G : Mentioned, no specification",H : Not mentioned,I : C3 Objectives - benefits of plastics,...,AI : Not mentioned,AJ : Sustainable production,AK : C8 Value chain,AL : 1. Upstream,AM : 2. Midstream,AN : 3. Downstream,AO : 4. Cross value chain,AP : C9 Type of measure,AQ : Instrument,AR : Target
0,1 : EU,True,False,True,False,True,True,False,False,True,...,False,False,True,True,True,True,True,True,True,True
1,2 : Bosnia and Herzegovina,True,False,True,False,True,True,False,False,True,...,False,False,True,True,True,True,True,True,True,False
2,3 : Brazil,True,False,True,False,True,False,True,False,False,...,True,False,True,True,True,True,False,True,True,False
3,4 : Cambodia,True,True,False,False,True,False,True,False,False,...,True,False,True,True,True,True,False,True,True,False
4,5 : Cook Islands,True,False,True,False,True,True,False,False,False,...,True,False,True,True,True,True,True,True,True,True
5,6 : Iran,True,False,True,False,False,False,False,True,False,...,True,False,True,False,False,True,False,True,True,False
6,7 : Monaco,True,True,False,False,True,True,False,False,False,...,True,False,True,True,True,True,True,True,True,True
7,8 : Russia,True,False,True,False,False,False,False,True,True,...,True,False,True,False,False,True,False,True,True,True
8,9 : Saudi Arabia,True,False,True,False,True,False,True,False,True,...,True,False,True,False,True,True,False,True,True,True
9,10 : Uruguay,True,False,True,False,True,True,False,False,False,...,True,False,True,True,True,True,True,True,True,True


## Match columns and perform IRR calculation

In [35]:
llm_data

,country,A : C1 Objectives - end plastic pollution,B : Mentioned with time frame,"C : Mentioned, no time frame",D : Not mentioned,E : C2 Objectives - reduce production of plastics,F : Mentioned with specification,"G : Mentioned, no specification",H : Not mentioned,I : C3 Objectives - benefits of plastics,...,AI : Not mentioned,AJ : Sustainable production,AK : C8 Value chain,AL : 1. Upstream,AM : 2. Midstream,AN : 3. Downstream,AO : 4. Cross value chain,AP : C9 Type of measure,AQ : Instrument,AR : Target
0,1 : EU,True,False,True,False,True,True,False,False,True,...,False,False,True,True,True,True,True,True,True,True
1,2 : Bosnia and Herzegovina,True,False,True,False,True,True,False,False,True,...,False,False,True,True,True,True,True,True,True,False
2,3 : Brazil,True,False,True,False,True,False,True,False,False,...,True,False,True,True,True,True,False,True,True,False
3,4 : Cambodia,True,True,False,False,True,False,True,False,False,...,True,False,True,True,True,True,False,True,True,False
4,5 : Cook Islands,True,False,True,False,True,True,False,False,False,...,True,False,True,True,True,True,True,True,True,True
5,6 : Iran,True,False,True,False,False,False,False,True,False,...,True,False,True,False,False,True,False,True,True,False
6,7 : Monaco,True,True,False,False,True,True,False,False,False,...,True,False,True,True,True,True,True,True,True,True
7,8 : Russia,True,False,True,False,False,False,False,True,True,...,True,False,True,False,False,True,False,True,True,True
8,9 : Saudi Arabia,True,False,True,False,True,False,True,False,True,...,True,False,True,False,True,True,False,True,True,True
9,10 : Uruguay,True,False,True,False,True,True,False,False,False,...,True,False,True,True,True,True,True,True,True,True


In [70]:
import pandas as pd
import numpy as np
import re

def calculate_category_irr(df1, df2, category_mappings):
    """
    Calculate Gwet's AC1 at the category level by aggregating related columns.
    
    Parameters:
    -----------
    df1 : pandas.DataFrame
        First dataframe (e.g., cleaned LLM data)
    df2 : pandas.DataFrame
        Second dataframe (e.g., cleaned NVivo data)
    category_mappings : dict
        Dictionary mapping category names to lists of column names
    
    Returns:
    --------
    dict
        Dictionary containing Gwet's AC1 scores for each category
    """
    # Make a copy to avoid modifying the original dataframes
    df1_copy = df1.copy()
    df2_copy = df2.copy()
    
    # Extract numeric index from country names for proper sorting
    def extract_country_index(country_str):
        """Extract the numeric index from country strings like '10 : Uruguay'"""
        try:
            # Extract the number at the beginning of the string
            import re
            match = re.match(r'^(\d+)', str(country_str))
            if match:
                return int(match.group(1))
            return 999  # Fallback for countries without numeric prefix
        except:
            return 999  # Fallback value
    
    # Create a new column with the numeric index
    df1_copy['country_index'] = df1_copy['country'].apply(extract_country_index)
    df2_copy['country_index'] = df2_copy['country'].apply(extract_country_index)
    
    # Sort by the numeric index 
    df1_copy = df1_copy.sort_values('country_index')
    df2_copy = df2_copy.sort_values('country_index')
    
    # Verify alignment
    if df1_copy['country'].tolist() != df2_copy['country'].tolist():
        print("\n⚠️ WARNING: Country lists don't match after sorting!")
        print("Using direct position-based alignment instead.")
        
        # Create a standardized reference list from the first dataframe
        reference_countries = df1_copy['country'].tolist()
        
        # Reindex the second dataframe to match the first
        country_order_mapping = {country: i for i, country in enumerate(reference_countries)}
        df2_positions = [country_order_mapping.get(country, 999) for country in df2_copy['country']]
        df2_copy['_position'] = df2_positions
        df2_copy = df2_copy.sort_values('_position').drop('_position', axis=1)
    
    # Clean up temporary columns
    df1_copy = df1_copy.drop('country_index', axis=1)
    df2_copy = df2_copy.drop('country_index', axis=1)
    
    if df1_copy['country'].tolist() == df2_copy['country'].tolist():
        print("✓ Countries are perfectly aligned!")
    
    # Create aggregated category columns
    results = {}
    
    for category, columns in category_mappings.items():
        # Verify columns exist in the dataframes
        existing_columns = [col for col in columns if col in df1_copy.columns and col in df2_copy.columns]
        
        if not existing_columns:
            print(f"Warning: No columns found for category {category}")
            continue
            
        print(f"\nProcessing category: {category}")
        print(f"Using columns: {existing_columns}")
        
        # Encode the category value for each row in each dataframe based on column values
        df1_category_values = []
        df2_category_values = []
        
        for idx in range(len(df1_copy)):
            # Check if any column value is True
            df1_row_has_true = any(df1_copy.iloc[idx][col] for col in existing_columns if col in df1_copy.columns)
            df2_row_has_true = any(df2_copy.iloc[idx][col] for col in existing_columns if col in df2_copy.columns)
            
            df1_category_values.append(1 if df1_row_has_true else 0)
            df2_category_values.append(1 if df2_row_has_true else 0)
        
        # Calculate Gwet's AC1 for this category
        try:
            ac1 = gwets_ac1_manual(df1_category_values, df2_category_values)
            results[category] = ac1
            print(f"  Gwet's AC1: {ac1:.4f}")
        except Exception as e:
            results[category] = f"Error: {str(e)}"
            print(f"  Error: {str(e)}")
    
    return results

def gwets_ac1_manual(ratings1, ratings2):
    """
    Manually calculate Gwet's AC1 coefficient between two raters.
    
    Parameters:
    -----------
    ratings1 : numpy.ndarray
        Ratings from first rater
    ratings2 : numpy.ndarray
        Ratings from second rater
    
    Returns:
    --------
    float
        Gwet's AC1 coefficient
    """
    # Ensure ratings are numeric
    ratings1 = np.array(ratings1, dtype=float)
    ratings2 = np.array(ratings2, dtype=float)
    
    # Number of subjects
    n = len(ratings1)
    
    # Calculate observed agreement
    agreement = (ratings1 == ratings2).sum() / n
    
    # Calculate probability of chance agreement (specific to Gwet's AC1)
    # For binary ratings (0/1 or True/False)
    p1 = (ratings1.mean() + ratings2.mean()) / 2
    p_e = 2 * p1 * (1 - p1)
    
    # Calculate Gwet's AC1
    ac1 = (agreement - p_e) / (1 - p_e)
    
    return ac1

def clean_datasets_for_irr(llm_data, nvivo_data):
    """
    Clean LLM and NVivo datasets by removing category header columns to prepare for IRR calculation.
    
    Parameters:
    -----------
    llm_data : pandas.DataFrame
        The dataframe containing LLM coding results
    nvivo_data : pandas.DataFrame
        The dataframe containing NVivo coding results
    
    Returns:
    --------
    tuple
        (cleaned_llm_data, cleaned_nvivo_data) - Dataframes with category header columns removed
    """
    # Create copies to avoid modifying the original dataframes
    llm_clean = llm_data.copy()
    nvivo_clean = nvivo_data.copy()
    
    # Identify category header columns based on pattern matching (C1, C2, C3, etc.)
    # These are the columns we want to exclude
    category_columns = []
    
    for col in llm_data.columns:
        # Match columns that contain "C" followed by a number in their name
        if re.search(r'C\d+\s+Objectives|C\d+\s+Time|C\d+\s+Stringency|C\d+\s+Value|C\d+\s+Type', col):
            category_columns.append(col)
    
    # Print identified category columns for verification
    print(f"Identified {len(category_columns)} category columns to exclude:")
    for col in category_columns:
        print(f"  - {col}")
    
    # Drop the category columns from both dataframes
    llm_clean = llm_clean.drop(columns=category_columns)
    nvivo_clean = nvivo_clean.drop(columns=category_columns)
    
    # Verify both dataframes have the same columns left
    remaining_columns = llm_clean.columns
    print(f"\nRemaining columns for IRR calculation: {len(remaining_columns)}")
    
    # Make sure both dataframes have identical columns
    assert set(llm_clean.columns) == set(nvivo_clean.columns), "Column mismatch after cleaning"
    
    return llm_clean, nvivo_clean

def calculate_gwets_ac1(df1, df2):
    """
    Calculate Gwet's AC1 coefficient for inter-rater reliability between two dataframes
    using a manual implementation.
    
    Parameters:
    -----------
    df1 : pandas.DataFrame
        First dataframe (e.g., cleaned LLM data)
    df2 : pandas.DataFrame
        Second dataframe (e.g., cleaned NVivo data)
    
    Returns:
    --------
    dict
        Dictionary containing Gwet's AC1 scores for each column
    """
    # Make a copy to avoid modifying the original dataframes
    df1_copy = df1.copy()
    df2_copy = df2.copy()
    
    # Extract numeric index from country names for proper sorting
    def extract_country_index(country_str):
        """Extract the numeric index from country strings like '10 : Uruguay'"""
        try:
            # Extract the number at the beginning of the string
            import re
            match = re.match(r'^(\d+)', str(country_str))
            if match:
                return int(match.group(1))
            return 999  # Fallback for countries without numeric prefix
        except:
            return 999  # Fallback value
    
    # Create a new column with the numeric index
    df1_copy['country_index'] = df1_copy['country'].apply(extract_country_index)
    df2_copy['country_index'] = df2_copy['country'].apply(extract_country_index)
    
    # Sort by the numeric index 
    df1_copy = df1_copy.sort_values('country_index')
    df2_copy = df2_copy.sort_values('country_index')
    
    # Verify ordering
    print("Countries in first dataset after sorting:")
    for country in df1_copy['country'].tolist():
        print(f"  {country}")
    
    print("\nCountries in second dataset after sorting:")
    for country in df2_copy['country'].tolist():
        print(f"  {country}")
    
    # Verify alignment
    if df1_copy['country'].tolist() != df2_copy['country'].tolist():
        print("\n⚠️ WARNING: Country lists still don't match after sorting!")
        print("Using direct position-based alignment instead.")
        
        # Create a standardized reference list from the first dataframe
        reference_countries = df1_copy['country'].tolist()
        
        # Reindex the second dataframe to match the first
        country_order_mapping = {country: i for i, country in enumerate(reference_countries)}
        df2_positions = [country_order_mapping.get(country, 999) for country in df2_copy['country']]
        df2_copy['_position'] = df2_positions
        df2_copy = df2_copy.sort_values('_position').drop('_position', axis=1)
    
    # Clean up temporary columns
    df1_copy = df1_copy.drop('country_index', axis=1)
    df2_copy = df2_copy.drop('country_index', axis=1)
    
    # One final verification
    print("\nChecking final alignment...")
    for c1, c2 in zip(df1_copy['country'].tolist(), df2_copy['country'].tolist()):
        if c1 != c2:
            print(f"  Mismatch: {c1} vs {c2}")
    
    if df1_copy['country'].tolist() == df2_copy['country'].tolist():
        print("  ✓ Countries are perfectly aligned!")
    
    results = {}
    
    # Calculate Gwet's AC1 for each column EXCEPT 'country'
    for col in df1_copy.columns:
        # Skip the country and index columns
        if col == 'country' or col == 'country_index':
            continue
            
        # Extract ratings as numpy arrays
        ratings1 = df1_copy[col].values
        ratings2 = df2_copy[col].values
        
        # Skip non-numeric columns
        try:
            # Convert boolean values to integers if needed
            if ratings1.dtype == bool:
                ratings1 = ratings1.astype(int)
            if ratings2.dtype == bool:
                ratings2 = ratings2.astype(int)
            
            # Calculate Gwet's AC1 using our manual implementation
            ac1 = gwets_ac1_manual(ratings1, ratings2)
            results[col] = ac1
        except (ValueError, TypeError) as e:
            # Skip columns that can't be converted to numeric values
            print(f"Skipping column '{col}' - not numeric: {str(e)}")
        except Exception as e:
            results[col] = f"Error: {str(e)}"
            
    return results

def analyze_irr(llm_data, nvivo_data):
    """
    Analyze inter-rater reliability between LLM and NVivo data at the category level.
    
    Parameters:
    -----------
    llm_data : pandas.DataFrame
        The dataframe containing LLM coding results
    nvivo_data : pandas.DataFrame
        The dataframe containing NVivo coding results
    
    Returns:
    --------
    tuple
        (cleaned_llm_data, cleaned_nvivo_data, irr_results)
    """
    print(f"LLM data: {llm_data.shape[0]} rows and {llm_data.shape[1]} columns")
    print(f"NVivo data: {nvivo_data.shape[0]} rows and {nvivo_data.shape[1]} columns")
    
    # Clean the datasets (remove category header columns)
    llm_clean, nvivo_clean = clean_datasets_for_irr(llm_data, nvivo_data)
    
    # Define the category mappings based on column groups
    category_mappings = {
        'C1: End plastic pollution': ['B : Mentioned with time frame', 'C : Mentioned, no time frame', 'D : Not mentioned'],
        'C2: Reduce production of plastics': ['F : Mentioned with specification', 'G : Mentioned, no specification', 'H : Not mentioned'],
        'C3: Benefits of plastics': ['J : Mentioned', 'K : Not mentioned'],
        'C4: Protect human health': ['M : Mentioned', 'N : Not mentioned'],
        'C5: Protect biodiversity and environment': ['P : Mentioned', 'Q : Not mentioned'],
        'C10: Time horizon of implementation': ['S : Not relevant', 'T : Not specified', 'U : Specified'],
        'C11: Stringency of measure': ['W : High', 'X : Low', 'Y : Non relevant'],
        'C6: Addressing full life cycle': ['AA : Mentioned', 'AB : Not mentioned', 'AC : Partial mention'],
        'C7: Other objectives': ['AE : Circular economy', 'AF : Climate change', 'AG : ESM', 
                                'AH : Mentioned', 'AI : Not mentioned', 'AJ : Sustainable production'],
        'C8: Value chain': ['AL : 1. Upstream', 'AM : 2. Midstream', 'AN : 3. Downstream', 'AO : 4. Cross value chain'],
        'C9: Type of measure': ['AQ : Instrument', 'AR : Target']
    }
    
    # Calculate Gwet's AC1 at the category level
    category_results = calculate_category_irr(llm_clean, nvivo_clean, category_mappings)
    
    # Display results
    print("\nGwet's AC1 results by coding category:")
    for category, ac1 in category_results.items():
        if isinstance(ac1, float):
            print(f"{category}: {ac1:.4f}")
        else:
            print(f"{category}: {ac1}")
    
    # Calculate overall average AC1
    valid_scores = [score for score in category_results.values() if isinstance(score, float)]
    if valid_scores:
        avg_ac1 = sum(valid_scores) / len(valid_scores)
        print(f"\nAverage Gwet's AC1 across all categories: {avg_ac1:.4f}")
        print(f"Number of categories included in average: {len(valid_scores)}")
    else:
        print("\nNo valid AC1 scores calculated.")
    
    return llm_clean, nvivo_clean, category_results
    
    return llm_clean, nvivo_clean, irr_results

In [56]:
llm_data.country

0                         1 : EU
1     2 : Bosnia and Herzegovina
2                     3 : Brazil
3                   4 : Cambodia
4               5 : Cook Islands
5                       6 : Iran
6                     7 : Monaco
7                     8 : Russia
8               9 : Saudi Arabia
9                   10 : Uruguay
10                      11 : USA
Name: country, dtype: category
Categories (11, object): ['1 : EU' < '2 : Bosnia and Herzegovina' < '3 : Brazil' < '4 : Cambodia' ... '8 : Russia' < '9 : Saudi Arabia' < '10 : Uruguay' < '11 : USA']

In [57]:
check_nvivo.country

0                         1 : EU
1     2 : Bosnia and Herzegovina
2                     3 : Brazil
3                   4 : Cambodia
4               5 : Cook Islands
5                       6 : Iran
6                     7 : Monaco
7                     8 : Russia
8               9 : Saudi Arabia
9                   10 : Uruguay
10                      11 : USA
Name: country, dtype: object

In [71]:
llm_clean, nvivo_clean, irr_results = analyze_irr(llm_data, check_nvivo)

print("\nExample of cleaned data:")
print(llm_clean.head())

LLM data: 11 rows and 45 columns
NVivo data: 11 rows and 45 columns
Identified 11 category columns to exclude:
  - A : C1 Objectives - end plastic pollution
  - E : C2 Objectives - reduce production of plastics
  - I : C3 Objectives - benefits of plastics
  - L : C4 Objectives - protect human health
  - O : C5 Objectives - protect biodiversity and (marine) environment
  - R : C10 Time horizon of implementation
  - V : C11 Stringency of measure
  - Z : C6 Objectives - addressing the full life cycle of plastics
  - AD : C7 Objectives - other objectives
  - AK : C8 Value chain
  - AP : C9 Type of measure

Remaining columns for IRR calculation: 34
✓ Countries are perfectly aligned!

Processing category: C1: End plastic pollution
Using columns: ['B : Mentioned with time frame', 'C : Mentioned, no time frame', 'D : Not mentioned']
  Gwet's AC1: 1.0000

Processing category: C2: Reduce production of plastics
Using columns: ['F : Mentioned with specification', 'G : Mentioned, no specification',

In [68]:
check_nvivo.columns

Index(['country', 'A : C1 Objectives - end plastic pollution',
       'B : Mentioned with time frame', 'C : Mentioned, no time frame',
       'D : Not mentioned',
       'E : C2 Objectives - reduce production of plastics',
       'F : Mentioned with specification', 'G : Mentioned, no specification',
       'H : Not mentioned', 'I : C3 Objectives - benefits of plastics',
       'J : Mentioned', 'K : Not mentioned',
       'L : C4 Objectives - protect human health', 'M : Mentioned',
       'N : Not mentioned',
       'O : C5 Objectives - protect biodiversity and (marine) environment',
       'P : Mentioned', 'Q : Not mentioned',
       'R : C10 Time horizon of implementation', 'S : Not relevant',
       'T : Not specified', 'U : Specified', 'V : C11 Stringency of measure',
       'W : High', 'X : Low', 'Y : Non relevant',
       'Z : C6 Objectives - addressing the full life cycle of plastics',
       'AA : Mentioned', 'AB : Not mentioned', 'AC : Partial mention',
       'AD : C7 Objectiv

In [76]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

def check_category_agreement(llm_data, nvivo_data, category_mappings=None):
    """
    Manually check agreement between two dataframes category by category.
    
    Parameters:
    -----------
    llm_data : pandas.DataFrame
        The dataframe containing LLM coding results
    nvivo_data : pandas.DataFrame
        The dataframe containing NVivo coding results
    category_mappings : dict, optional
        Dictionary mapping category names to lists of column names
        
    Returns:
    --------
    pandas.DataFrame
        Comparison dataframe showing category values and agreement
    """
    # If no category mappings provided, use the default
    if category_mappings is None:
        category_mappings = {
            'C1: End plastic pollution': ['B : Mentioned with time frame', 'C : Mentioned, no time frame', 'D : Not mentioned'],
            'C2: Reduce production of plastics': ['F : Mentioned with specification', 'G : Mentioned, no specification', 'H : Not mentioned'],
            'C3: Benefits of plastics': ['J : Mentioned', 'K : Not mentioned'],
            'C4: Protect human health': ['M : Mentioned', 'N : Not mentioned'],
            'C5: Protect biodiversity and environment': ['P : Mentioned', 'Q : Not mentioned'],
            'C10: Time horizon of implementation': ['S : Not relevant', 'T : Not specified', 'U : Specified'],
            'C11: Stringency of measure': ['W : High', 'X : Low', 'Y : Non relevant'],
            'C6: Addressing full life cycle': ['AA : Mentioned', 'AB : Not mentioned', 'AC : Partial mention'],
            'C7: Other objectives': ['AE : Circular economy', 'AF : Climate change', 'AG : ESM', 
                                    'AH : Mentioned', 'AI : Not mentioned', 'AJ : Sustainable production'],
            'C8: Value chain': ['AL : 1. Upstream', 'AM : 2. Midstream', 'AN : 3. Downstream', 'AO : 4. Cross value chain'],
            'C9: Type of measure': ['AQ : Instrument', 'AR : Target']
        }
    
    # Ensure both dataframes have the same order of countries
    llm_sorted = llm_data.sort_values('country').reset_index(drop=True)
    nvivo_sorted = nvivo_data.sort_values('country').reset_index(drop=True)
    
    # Create a comparison dataframe
    comparison_data = []
    
    # Process each row (country)
    for i in range(len(llm_sorted)):
        country = llm_sorted.iloc[i]['country']
        row_data = {'country': country}
        
        # Process each category
        for category, columns in category_mappings.items():
            # Check which columns exist in both dataframes
            existing_columns = [col for col in columns if col in llm_sorted.columns and col in nvivo_sorted.columns]
            
            if not existing_columns:
                continue
                
            # Check if any column is True for each dataframe
            llm_has_true = any(llm_sorted.iloc[i][col] for col in existing_columns)
            nvivo_has_true = any(nvivo_sorted.iloc[i][col] for col in existing_columns)
            
            # Store the values and agreement
            row_data[f"{category} (LLM)"] = llm_has_true
            row_data[f"{category} (NVivo)"] = nvivo_has_true
            row_data[f"{category} (Match)"] = llm_has_true == nvivo_has_true
        
        comparison_data.append(row_data)
    
    # Create the comparison DataFrame
    comparison_df = pd.DataFrame(comparison_data)
    
    # Calculate summary statistics
    print("=== CATEGORY AGREEMENT SUMMARY ===")
    summary_data = []
    
    for category in category_mappings.keys():
        match_column = f"{category} (Match)"
        if match_column in comparison_df.columns:
            matches = comparison_df[match_column].sum()
            total = len(comparison_df)
            percent_agreement = (matches / total) * 100
            
            # Get the actual values for both raters
            llm_true_count = comparison_df[f"{category} (LLM)"].sum()
            nvivo_true_count = comparison_df[f"{category} (NVivo)"].sum()
            
            summary_data.append({
                'Category': category,
                'Matches': matches,
                'Total': total,
                'Percent Agreement': f"{percent_agreement:.2f}%",
                'LLM True Count': llm_true_count,
                'NVivo True Count': nvivo_true_count,
                'Value Difference': abs(llm_true_count - nvivo_true_count)
            })
    
    summary_df = pd.DataFrame(summary_data)
    print(summary_df)
    
    # Check for categories with perfect agreement
    perfect_agreement = summary_df[summary_df['Percent Agreement'] == '100.00%']
    print(f"\nNumber of categories with perfect agreement: {len(perfect_agreement)} out of {len(summary_df)}")
    
    # Check for categories where both raters always assigned the same value
    same_value_categories = summary_df[(summary_df['LLM True Count'] == 0) & (summary_df['NVivo True Count'] == 0) | 
                                      (summary_df['LLM True Count'] == summary_df['Total']) & 
                                      (summary_df['NVivo True Count'] == summary_df['Total'])]
    
    if not same_value_categories.empty:
        print("\nCategories where both raters assigned the same value to ALL countries:")
        for _, row in same_value_categories.iterrows():
            value = "ALL TRUE" if row['LLM True Count'] == row['Total'] else "ALL FALSE"
            print(f"  - {row['Category']}: {value}")
    
    # Detailed view of disagreements
    print("\n=== DISAGREEMENT DETAILS ===")
    for category in category_mappings.keys():
        match_column = f"{category} (Match)"
        if match_column in comparison_df.columns:
            disagreements = comparison_df[~comparison_df[match_column]]
            if not disagreements.empty:
                print(f"\nDisagreements for {category}:")
                for _, row in disagreements.iterrows():
                    print(f"  {row['country']}: LLM={row[f'{category} (LLM)']}, NVivo={row[f'{category} (NVivo)']}")
    
    # Return the full comparison dataframe for further analysis
    return comparison_df

def visualize_category_agreement(comparison_df, category_mappings=None):
    """
    Visualize the agreement between raters across categories.
    
    Parameters:
    -----------
    comparison_df : pandas.DataFrame
        The comparison dataframe from check_category_agreement
    category_mappings : dict, optional
        Dictionary mapping category names to lists of column names
    """
    if category_mappings is None:
        # Extract categories from column names
        categories = [col.split(' (')[0] for col in comparison_df.columns if ' (Match)' in col]
    else:
        categories = list(category_mappings.keys())
    
    # Calculate agreement percentages
    agreement_data = []
    
    for category in categories:
        match_column = f"{category} (Match)"
        if match_column in comparison_df.columns:
            matches = comparison_df[match_column].sum()
            total = len(comparison_df)
            percent_agreement = (matches / total) * 100
            agreement_data.append({
                'Category': category,
                'Agreement': percent_agreement
            })
    
    # Create bar chart
    if agreement_data:
        agreement_df = pd.DataFrame(agreement_data)
        
        plt.figure(figsize=(12, 6))
        bars = plt.bar(agreement_df['Category'], agreement_df['Agreement'])
        
        # Add a horizontal line at 100%
        plt.axhline(y=100, color='r', linestyle='--', alpha=0.7)
        
        # Color bars based on agreement level
        for i, bar in enumerate(bars):
            if agreement_df.iloc[i]['Agreement'] == 100:
                bar.set_color('green')
            elif agreement_df.iloc[i]['Agreement'] >= 80:
                bar.set_color('yellowgreen')
            elif agreement_df.iloc[i]['Agreement'] >= 60:
                bar.set_color('gold')
            else:
                bar.set_color('tomato')
        
        plt.xlabel('Category')
        plt.ylabel('Agreement (%)')
        plt.title('Inter-Rater Agreement by Category')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.ylim(0, 105)
        
        for i, v in enumerate(agreement_df['Agreement']):
            plt.text(i, v + 2, f"{v:.1f}%", ha='center', fontsize=9)
        
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()
    
    return agreement_data

# Example usage:
"""
# Load your data
llm_data = pd.read_csv("path_to_llm_data.csv")
nvivo_data = pd.read_csv("path_to_nvivo_data.csv")

# Check agreement
comparison_df = check_category_agreement(llm_data, nvivo_data)

# Visualize agreement
visualize_category_agreement(comparison_df)
"""

'\n# Load your data\nllm_data = pd.read_csv("path_to_llm_data.csv")\nnvivo_data = pd.read_csv("path_to_nvivo_data.csv")\n\n# Check agreement\ncomparison_df = check_category_agreement(llm_data, nvivo_data)\n\n# Visualize agreement\nvisualize_category_agreement(comparison_df)\n'

In [77]:
comparison_df = check_category_agreement(llm_data, check_nvivo)

=== CATEGORY AGREEMENT SUMMARY ===
                                    Category  Matches  Total  \
0                  C1: End plastic pollution       11     11   
1          C2: Reduce production of plastics       11     11   
2                   C3: Benefits of plastics       11     11   
3                   C4: Protect human health       11     11   
4   C5: Protect biodiversity and environment       11     11   
5        C10: Time horizon of implementation        1     11   
6                 C11: Stringency of measure        3     11   
7             C6: Addressing full life cycle       11     11   
8                       C7: Other objectives        4     11   
9                            C8: Value chain       11     11   
10                       C9: Type of measure        4     11   

   Percent Agreement  LLM True Count  NVivo True Count  Value Difference  
0            100.00%              11                11                 0  
1            100.00%              11          

In [ ]:
import pandas as pd
import numpy as np

def check_column_agreement(llm_data, nvivo_data):
    """
    Calculate agreement percentage for each individual column between two dataframes.
    
    Parameters:
    -----------
    llm_data : pandas.DataFrame
        The dataframe containing LLM coding results
    nvivo_data : pandas.DataFrame
        The dataframe containing NVivo coding results
        
    Returns:
    --------
    pandas.DataFrame
        Summary dataframe showing column-by-column agreement statistics
    """
    # Ensure both dataframes have the same order of countries
    # First, convert category type to string if needed
    if llm_data['country'].dtype.name == 'category':
        llm_data = llm_data.copy()
        llm_data['country'] = llm_data['country'].astype(str)
    if nvivo_data['country'].dtype.name == 'category':
        nvivo_data = nvivo_data.copy()
        nvivo_data['country'] = nvivo_data['country'].astype(str)
    
    # Sort both dataframes by country
    llm_sorted = llm_data.sort_values('country').reset_index(drop=True)
    nvivo_sorted = nvivo_data.sort_values('country').reset_index(drop=True)
    
    # Verify countries are aligned
    if not all(llm_sorted['country'].values == nvivo_sorted['country'].values):
        # Use a more robust numeric sorting for countries with numbers
        def extract_country_index(country_str):
            try:
                import re
                match = re.match(r'^(\d+)', str(country_str))
                if match:
                    return int(match.group(1))
                return 999
            except:
                return 999
        
        # Resort using numeric indices
        llm_sorted['country_index'] = llm_sorted['country'].apply(extract_country_index)
        nvivo_sorted['country_index'] = nvivo_sorted['country'].apply(extract_country_index)
        
        llm_sorted = llm_sorted.sort_values('country_index').reset_index(drop=True)
        nvivo_sorted = nvivo_sorted.sort_values('country_index').reset_index(drop=True)
        
        # Remove temporary column
        llm_sorted = llm_sorted.drop('country_index', axis=1)
        nvivo_sorted = nvivo_sorted.drop('country_index', axis=1)
    
    # Verify countries are aligned now
    assert all(llm_sorted['country'].values == nvivo_sorted['country'].values), "Countries still not aligned after sorting!"
    
    # Get common columns (excluding 'country')
    common_columns = [col for col in llm_sorted.columns if col in nvivo_sorted.columns and col != 'country']
    
    # Store results
    results = []
    
    # Check each column
    for col in common_columns:
        # Skip if not boolean/numeric data
        if not pd.api.types.is_bool_dtype(llm_sorted[col]) and not pd.api.types.is_numeric_dtype(llm_sorted[col]):
            continue
            
        # Convert to boolean for comparison if needed
        llm_col = llm_sorted[col].astype(bool)
        nvivo_col = nvivo_sorted[col].astype(bool)
        
        # Calculate agreement
        matches = (llm_col == nvivo_col).sum()
        total = len(llm_col)
        percent_agreement = (matches / total) * 100
        
        # Count True values in each dataset
        llm_true_count = llm_col.sum()
        nvivo_true_count = nvivo_col.sum()
        
        # Check if both all True or all False
        all_same_llm = (llm_true_count == 0) or (llm_true_count == total)
        all_same_nvivo = (nvivo_true_count == 0) or (nvivo_true_count == total)
        
        # Identify the pattern (all True, all False, or mixed)
        llm_pattern = "ALL TRUE" if llm_true_count == total else "ALL FALSE" if llm_true_count == 0 else "MIXED"
        nvivo_pattern = "ALL TRUE" if nvivo_true_count == total else "ALL FALSE" if nvivo_true_count == 0 else "MIXED"
        
        # Prepare detailed disagreement info
        disagreements = []
        if matches < total:
            for i in range(total):
                if llm_col.iloc[i] != nvivo_col.iloc[i]:
                    disagreements.append(f"{llm_sorted['country'].iloc[i]}: LLM={llm_col.iloc[i]}, NVivo={nvivo_col.iloc[i]}")
        
        # Store the results
        results.append({
            'Column': col,
            'Agreement %': round(percent_agreement, 2),
            'Matches': matches,
            'Total': total,
            'LLM True Count': llm_true_count,
            'NVivo True Count': nvivo_true_count,
            'LLM Pattern': llm_pattern,
            'NVivo Pattern': nvivo_pattern,
            'Both All Same': all_same_llm and all_same_nvivo,
            'Disagreements': "; ".join(disagreements) if disagreements else "None"
        })
    
    # Convert to DataFrame and sort by agreement
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Agreement %', ascending=False)
    
    # Summary statistics
    perfect_agreement = results_df[results_df['Agreement %'] == 100]
    all_same_pattern = results_df[results_df['Both All Same'] == True]
    
    print(f"=== COLUMN AGREEMENT SUMMARY ===")
    print(f"Total columns analyzed: {len(results_df)}")
    print(f"Columns with 100% agreement: {len(perfect_agreement)} ({len(perfect_agreement)/len(results_df)*100:.1f}%)")
    print(f"Columns where both datasets have all TRUE or all FALSE: {len(all_same_pattern)} ({len(all_same_pattern)/len(results_df)*100:.1f}%)")
    
    # Distribution of agreement levels
    agreement_bins = [(0, 60), (60, 80), (80, 90), (90, 99), (99, 100)]
    print("\nAgreement distribution:")
    for low, high in agreement_bins:
        count = len(results_df[(results_df['Agreement %'] >= low) & (results_df['Agreement %'] <= high)])
        print(f"  {low}-{high}%: {count} columns ({count/len(results_df)*100:.1f}%)")
    
    # Print columns with pattern "ALL TRUE" or "ALL FALSE" in both datasets
    print("\nColumns where both datasets have ALL TRUE or ALL FALSE:")
    for _, row in all_same_pattern.iterrows():
        print(f"  {row['Column']}: LLM={row['LLM Pattern']}, NVivo={row['NVivo Pattern']}")
    
    # Return the detailed results
    return results_df

# Example usage:
"""
# Load your data
llm_data = pd.read_csv("path_to_llm_data.csv")
nvivo_data = pd.read_csv("path_to_nvivo_data.csv")

# Check column-level agreement
column_agreement = check_column_agreement(llm_data, nvivo_data)

# Display detailed results for columns with less than 100% agreement
disagreements = column_agreement[column_agreement['Agreement %'] < 100]
if not disagreements.empty:
    print("\nDetailed disagreements:")
    for _, row in disagreements.iterrows():
        print(f"\n{row['Column']} ({row['Agreement %']}% agreement):")
        print(f"Disagreements: {row['Disagreements']}")
"""